### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

### Verificar se a biblioteca da OCI está baixada

In [36]:
!oci --version

3.54.2


##### Caso não esteja, rodar o comando:

!pip install oci

### Configure um arquivo com credenciais API Key
- No terminal rode o comado: 
> oci setup config
- Adicionar OCID (no seu perfil > user_setings)
- Adicionar Tenanty OCID (no seu perfil > tenancy)
- Deixar pastas padrão
- Configurar chave pública(perfil > user_settings > tokens and keys > add api key > paste a public key)

### Código direto do code examples no OCI GenAI
- Llama 3.3 70B 

⚠️⚠️⚠️⚠️⚠️ O código não funciona se você não alterar para o seu compartment_id e conferir model_id ⚠️⚠️⚠️⚠️⚠️⚠️

In [2]:
user_input = "O que é um impedimento?"

In [42]:
import oci

# Setup basic variables
# Auth Config
# TODO: Please update config profile name and use the compartmentId that has policies grant permissions for using Generative AI Service
compartment_id = "ocid1.compartment.oc1..aaaaaaaael2x4x2flfdgrteymct63e73zghik5gbyzd2zsww7ihzkfldpdza"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)

# Service endpoint
endpoint = "https://inference.generativeai.sa-saopaulo-1.oci.oraclecloud.com"

generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint)
chat_detail = oci.generative_ai_inference.models.ChatDetails()

content = oci.generative_ai_inference.models.TextContent()
content.text = f"{user_input}"
message = oci.generative_ai_inference.models.Message()
message.role = "USER"
message.content = [content]

chat_request = oci.generative_ai_inference.models.GenericChatRequest()
chat_request.api_format = oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC
chat_request.messages = [message]
chat_request.max_tokens = 600
chat_request.temperature = 1
chat_request.frequency_penalty = 0
chat_request.presence_penalty = 0
chat_request.top_p = 0.75

chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="ocid1.generativeaimodel.oc1.sa-saopaulo-1.amaaaaaask7dceyarsn4m6k3aqvvgatida3omyprlcs3alrwcuusblru4jaa")
chat_detail.chat_request = chat_request
chat_detail.compartment_id = compartment_id

chat_response = generative_ai_inference_client.chat(chat_detail)

# Print result
print("**************************Chat Result**************************")
choices = chat_response.data.chat_response.choices
if choices and len(choices) > 0:
    response_text = choices[0].message.content[0].text
else:
    response_text = "Nenhuma resposta foi gerada pelo modelo."
# Exibindo a resposta no console
print(json.dumps({"response": response_text}, indent=4, ensure_ascii=False))

**************************Chat Result**************************
{
    "response": "Um impedimento é uma circunstância ou condição que impede alguém de realizar uma ação, tomar uma decisão ou exercer uma função de forma eficaz. Pode ser um obstáculo, uma limitação ou uma restrição que dificulta ou impede o progresso, a realização de um objetivo ou a tomada de uma decisão.\n\nExistem vários tipos de impedimentos, incluindo:\n\n1. **Impedimentos físicos**: limitações físicas, como deficiências ou doenças, que impedem alguém de realizar certas atividades.\n2. **Impedimentos emocionais**: problemas emocionais, como ansiedade, depressão ou estresse, que afetam a capacidade de tomar decisões ou realizar tarefas.\n3. **Impedimentos cognitivos**: limitações cognitivas, como dificuldades de aprendizado ou memória, que impedem alguém de entender ou realizar certas tarefas.\n4. **Impedimentos sociais**: obstáculos sociais, como falta de apoio ou recursos, que impedem alguém de alcançar seus objeti

### Multimodal
- Llama 3.2 90B - processando imagens

In [40]:
# Configuração da mensagem do sistema e do usuário
system_msg = """Você é um especialista em futebol"""
user_msg = "quem é o jogador da imagem? "
image_content = "jogador1.png"

In [41]:
import oci
import base64
import json

# Configuração do cliente usando arquivo config
config = oci.config.from_file("~/.oci/config", "DEFAULT")
signer = oci.signer.Signer(
    tenancy=config["tenancy"],
    user=config["user"],
    fingerprint=config["fingerprint"],
    private_key_file_location=config["key_file"],
    pass_phrase=config.get("pass_phrase")
)
# Cliente para o serviço Generative AI
endpoint = "https://inference.generativeai.sa-saopaulo-1.oci.oraclecloud.com"
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config,
    signer=signer,
    service_endpoint=endpoint,
    retry_strategy=oci.retry.NoneRetryStrategy(),
    timeout=(100, 240)
)

#base 64 encode
try:
    with open(image_content, "rb") as image_file:
        image_data = image_file.read()
    base64_image = base64.b64encode(image_data).decode("utf-8")
    base64_image_url = f"data:image/jpeg;base64,{base64_image}"
except oci.exceptions.ServiceError as e:
    raise ValueError(f"Erro ao acessar a imagem")
    


# Configuração do conteúdo da imagem
imagem = oci.generative_ai_inference.models.ImageContent()
image_url_content = oci.generative_ai_inference.models.ImageUrl()
image_url_content.url = base64_image_url
imagem.image_url = image_url_content

# Configuração do conteúdo de texto
texto = oci.generative_ai_inference.models.TextContent()
texto.text = system_msg + user_msg

# Configuração da mensagem do usuário
message = oci.generative_ai_inference.models.UserMessage()
message.role = "USER"
message.content = [texto, imagem]


# Configuração da solicitação de chat
chat_request = oci.generative_ai_inference.models.GenericChatRequest()
chat_request.api_format = oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC
chat_request.messages = [message]
chat_request.num_generations = 1
chat_request.is_stream = False
chat_request.max_tokens = 4000
chat_request.temperature = 0.7
chat_request.frequency_penalty = 0
chat_request.presence_penalty = 0
chat_request.top_p = 0.5
chat_request.top_k = 1


# Configuração do chat_detail
chat_detail = oci.generative_ai_inference.models.ChatDetails()
chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(
    model_id="meta.llama-3.2-90b-vision-instruct"
)
chat_detail.chat_request = chat_request
chat_detail.compartment_id =  "ocid1.compartment.oc1..aaaaaaaael2x4x2flfdgrteymct63e73zghik5gbyzd2zsww7ihzkfldpdza"
# Chamada ao serviço
chat_response = generative_ai_inference_client.chat(chat_detail)
# Extração do texto da resposta
choices = chat_response.data.chat_response.choices
if choices and len(choices) > 0:
    response_text = choices[0].message.content[0].text
else:
    response_text = "Nenhuma resposta foi gerada pelo modelo."
# Exibindo a resposta no console
print(json.dumps({"response": response_text}, indent=4, ensure_ascii=False))

{
    "response": "O jogador da imagem é Cristiano Ronaldo, um dos maiores jogadores de futebol de todos os tempos. Ele é conhecido por sua habilidade excepcional, velocidade e precisão nos chutes. Ronaldo já jogou em times como Manchester United, Real Madrid e Juventus, e é um dos principais jogadores da seleção portuguesa. Ele é um dos jogadores mais premiados da história do futebol, tendo conquistado cinco Ballons d'Or e quatro títulos da UEFA Champions League."
}
